In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
from openai import OpenAI
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model="llama3.2"

# model="deepseek-r1:1.5b"

# response = ollama_via_openai.chat.completions.create(
#     model="deepseek-r1:1.5b",
#     messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
# )

# print(response.choices[0].message.content)

In [3]:
!ollama pull deepseek-r1:1.5b
!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest 
pulling aabd4debf0c8: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 GB                         
pulling c5ad996bda6e: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  556 B                         
pulling 6e4c38e1172f: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 KB                         
pulling f4d24e9138dd: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  148 B                         
pulling a85fe2a2e58e: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  487 B                         
verifying sha256 digest 
writing manifest 
suc

In [8]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# Custom exception for better granularity
class WebsiteError(Exception):
    """Base class for website‐related errors."""
    pass

class WebsiteNotFoundError(WebsiteError):
    """Raised on HTTP 404 Not Found."""
    pass

class WebsiteAccessDeniedError(WebsiteError):
    """Raised on HTTP 403 Forbidden (access denied)."""
    pass

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url, timeout=10):
        self.url = url
        try:
            # 2) Fetch with timeout
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            status = e.response.status_code
            # 3) Branch on status code
            if status == 404:
                raise WebsiteNotFoundError(f"404 Not Found: {url}") from e
            elif status == 403:
                raise WebsiteAccessDeniedError(f"403 Forbidden: {url}") from e
            else:
                # fallback for other 4xx/5xx
                raise WebsiteError(f"HTTP {status} error fetching {url}") from e
        except requests.exceptions.RequestException as e:
            # network issues, timeouts, DNS failures...
            raise WebsiteError(f"Connection error fetching {url}: {e}") from e
            

        # If we got here, response is good
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        
        # Title (guard against missing <title>)
        self.title = soup.title.string if soup.title else "No title found"
        
        # Extract and clean body text
        if soup.body:
            # Remove scripts, styles, images, inputs
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            # Get all text, preserving line breaks
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

# Ed Donner Site, testing of the function only

In [9]:
ed = Website("https://edwarddonner.com")
# ed.links

In [10]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
# print(link_system_prompt)

In [11]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
# print(get_links_user_prompt(ed))

In [13]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [14]:
anthropic = Website("https://anthropic.com")
# anthropic.links
anthropic.get_contents()

'Webpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nTeam plan\nEnterprise plan\nEducation plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certification\nLearn\nLearning resources\nCustomer stories\nEngineering at Anthropic\nAnthropic Academy\nCompany\nAbout\nCareers\nEvents\nEngineering\nBuilding effective agents\nNews\nEN\nThis is some text inside of a div 

In [15]:
# print(get_links_user_prompt(ed))

relevantlinks = get_links("https://anthropic.com")

In [16]:
# relevantlinks
for links in relevantlinks["links"]:
    print(links["url"])

# relevantlinks["links"]
# relevantlinks["links"]["url"]

https://www.anthropic.com/
https://www.anthropic.com/team
https://www.anthropic.com/company
https://www.anthropic.com/careers
https://docs.anthropic.com/
https://www_anthropic_com/contact-sales.html


# Testing and Checking only

In [17]:
# OLLAMA_API = "http://localhost:11434/api/chat"
# HEADERS = {"Content-Type": "application/json"}
# MODEL = "llama3.2"

# # Create a messages list using the same format that we used for OpenAI
# messages = [
#     {"role": "user", "content": "Describe some of the business applications of Generative AI"}
# ]

# payload = {
#         "model": MODEL,
#         "messages": messages,
#         "stream": False
#     }

# !ollama pull deepseek-r1:1.5b

# from openai import OpenAI
# ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

# response = ollama_via_openai.chat.completions.create(
#     model=MODEL,
#     messages=messages
# )

# print(response.choices[0].message.content)

# Part 2 from here onwards

In [18]:
def get_all_details(url):
    result = "Landing Page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result



In [19]:
get_all_details("https://anthropic.com")

Found links: {'links': [{'type': 'About page', 'url': 'https://www.anthropic.com/'}, {'type': 'Claude AI home page', 'url': 'https://claude.ai/'}, {'type': 'Employees and Team', 'url': 'https://www.anthropic.com/team'}, {'type': 'Enterprise Services', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'Education page', 'url': 'https://www.anthropic.com/education'}, {'type': 'Pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'API page', 'url': 'https://docs.anthropic.com/'}, {'type': 'Console access', 'url': 'https://console.anthropic.com/'}, {'type': 'Customer support solutions', 'url': 'https://www.anthropic.com/solutions/customer-support'}, {'type': 'Research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'Economic Index page', 'url': 'https://www.anthropic.com/economic-index'}, {'type': 'Claude AI code', 'url': 'https://claude.ai/download'}, {'type': 'Company news and blog', 'url': 'https://www.anthropic.com/news/'}, {'type': 'Engineering reso

'Landing Page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nTeam plan\nEnterprise plan\nEducation plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certification\nLearn\nLearning resources\nCustomer stories\nEngineering at Anthropic\nAnthropic Academy\nCompany\nAbout\nCareers\nEvents\nEngineering\nBuilding effective agents\nNews\nEN\nThis is some text i

In [20]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [21]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [22]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company overview', 'url': 'https://app.folio.com/company/the-hugging-face/About'}, {'type': 'pricing information', 'url': 'https://endpoints.huggingface.co/pricing#endpoints'}]}


WebsiteError: Connection error fetching https://app.folio.com/company/the-hugging-face/About: HTTPSConnectionPool(host='app.folio.com', port=443): Max retries exceeded with url: /company/the-hugging-face/About (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'app.folio.com'. (_ssl.c:1016)")))

In [ ]:
def create_brochure(company_name, url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=model,
        messages = [
            {"role" : "system", "content" : system_prompt},
            {"role" : "user", "content" : get_brochure_user_prompt("HuggingFace", "https://huggingface.co")}
        ],
        response_format = {"type": "json_object"}
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)